# Introduction
Rarely do we find ourselves demanding to know how predictive models do trivial things like determining the latest and greatest cat clip to put at the top of a Tik Tok feed. But for high stakes predictions or when conducting machine learning research, models cannot be treated as black boxes. In order to understand how input features affect model predictions, SHAP (SHapley Additive exPlanations), using theory by American mathematician Lloyd Shapley, make ML models interpretable.

In [2]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [64]:
from fugue_notebook import setup
setup()

<IPython.core.display.Javascript object>

# Load CSV Data
With the goal of focusing on Shapley values, we will be assuming data has been preprocessed. For convenience sake, we're using just a few numerical features with NA values removed.

In [88]:
import fugue.api as fa
from fugue.column import col, functions as f
import pprint as pp
import pandas as pd
import numpy as np

def drop_nas(df: pd.DataFrame) -> pd.DataFrame:
    df = df.replace("-99", None)
    df = df.dropna()
    return df


with fa.engine_context():
    df = fa.load("sps_abundance.csv", header=True, as_fugue=True)
    inputs = [f"Age {num + 1} Returns" for num in range(0 ,7)]
    output = "Number Of Spawners"
    df = fa.select_columns(df, inputs+[output])
    for coln in df.columns:
        fa.assign(df, coln=col(coln).cast("float"))
    df = fa.transform(df, drop_nas, schema="*")
    print(fa.as_pandas(df).describe(include="all"))
    returns_by_age = fa.select_columns(df, inputs)
    num_spawners = fa.select_columns(df, [output])
    # see if data looks right
    pp.pprint(fa.peek_dict(returns_by_age))
    pp.pprint(fa.peek_dict(num_spawners))
    

       Age 1 Returns Age 2 Returns Age 3 Returns Age 4 Returns Age 5 Returns  \
count          32446         32446         32446         32446         32446   
unique             9            84            97           101            99   
top                0             0             1             0             0   
freq           32202         25495         11850         12716         14666   

       Age 6 Returns Age 7 Returns Number Of Spawners  
count          32446         32446              32446  
unique            21             3               3012  
top                0             0                644  
freq           28061         32157                105  
{'Age 1 Returns': '0',
 'Age 2 Returns': '0',
 'Age 3 Returns': '.07',
 'Age 4 Returns': '.57',
 'Age 5 Returns': '.3',
 'Age 6 Returns': '.05',
 'Age 7 Returns': '.01'}
{'Number Of Spawners': '913'}


# Create Model   

In [89]:
from sklearn.linear_model import LinearRegression

X = returns_by_age.as_pandas()
y = num_spawners.as_pandas()
lin_reg = LinearRegression()
lin_reg = lin_reg.fit(X, y)

In [65]:
# define our predict function
def predict(df: pd.DataFrame, model: LinearRegression) -> pd.DataFrame:
    """
    Function to predict results using a pre-built model
    """
    return df.assign(predicted=model.predict(df))

# use Fugue transform to switch exection to spark
result = fa.transform(
    df=X,
    using=predict,
    schema="*,predicted:double",
    params=dict(model=lin_reg)
)

# result is a Spark DataFrame
print(type(result))
fa.show(fa.sample(result, n=6))

<class 'pandas.core.frame.DataFrame'>
PandasDataFrame
`Age 1 Returns`:str|`Age 2 Returns`:str|`Age 3 Returns`:str|`Age 4 Returns`:str|`Age 5 Returns`:str|`Age 6 Returns`:str|`Age 7 Returns`:str|predicted:double 
-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------------
0                  |0                  |0                  |.59                |.4                 |.01                |0                  |1350.427956761263
0                  |0                  |.09                |.91                |0                  |0                  |0                  |3240.031221289948
0                  |0                  |.06                |.48                |.42                |0                  |0                  |1773.004373186540
                   |                   |                   |                   |                   |                   |                   |8               

In [90]:
from itertools import cycle, permutations, chain
from collections import OrderedDict

def shapley_score(seed, rand_rows, row_index, model, feature_cols, weight_col_name):
    random_permutation = np.random.RandomState(seed).permutation(feature_cols)
    # We cycle through permutations in cases where the number of samples is more than
    # the number of features
    permutation_iter = cycle(permutations(random_permutation))
    feature_vector_rows = []
    rand_row_weights = []
    for rand_row in rand_rows:  # take sample z from training set
        rand_row_weights.append(rand_row[weight_col_name] if weight_col_name is not None else 1)
        feature_permutation = next(permutation_iter)  # choose permutation o
        # gather: {z_1, ..., z_p}
        feat_vec_without_feature = OrderedDict([(feat_name, rand_row[feat_name]) for feat_name in feature_cols])
        feat_vec_with_feature = feat_vec_without_feature.copy()
        for feat_name in feature_permutation:  # for random feature k.
            # x_+k = {x_1, ..., x_k, .. z_p}
            feat_vec_with_feature[feat_name] = row_index[feat_name]
            # x_-k = {x_1, ..., x_{k-1}, z_k, ..., z_p}
            # store (x_+k, x_-k)
            feature_vector_rows.append(feat_vec_with_feature.copy())
            feature_vector_rows.append(feat_vec_without_feature.copy())
            # (x_-k = x_+k)
            feat_vec_without_feature[feat_name] = row_index[feat_name]

    if len(feature_vector_rows) == 0:
        return
    preds = model.predict(feature_vector_rows)
    feature_iterator = chain.from_iterable(cycle(permutations(random_permutation)))
    for pred_index, feature in zip((range(0, len(preds), 2)), feature_iterator):
        marginal_contribution = preds[pred_index] - preds[pred_index + 1]

        #  There is one weight added per random row visited.
        #  For each random row visit, we generate 2 predictions for each required feature.
        #  Therefore, to get index into rand_row_weights, we need to divide
        #  prediction index by 2 * number of features, and take the floor of this.
        weight = rand_row_weights[pred_index // (len(feature_cols) * 2)]

        yield (str(feature), float(marginal_contribution), float(weight))

    

SyntaxError: invalid syntax (1928133087.py, line 1)